First Idea was to take faces and use face_recogition lib to compare retrieved face with premade set. It didn't workout as recognition was pretty bad. Next I tried to retriev faces and feed them to some clustering model like DBSCAN. It produced some good clusters, but they were not that great and several clusters were having a lot of crossovers and several persons would have several clusters consisting only of them. Then I tried to use facenet to improve DBSCAN and it was practicaly the same. However, using it for just embedding comparing was pretty good. In this notebook I only include DBSAN and facenet atempts.

In [1]:
%%capture
!pip install youtube_dl
!youtube-dl https://www.youtube.com/watch?v=gOFt-sz1T2A&feature=emb_title
!pip install face_recognition
!pip install scikit-learn
!pip install keras-facenet

## Clustering

It takes a long time to run as it first needs to collect faces (it takes up most of the time), then process them in cluster and then make output.

In [ ]:
embedder = FaceNet()
all_faces = []
faces_img_box = []
faces_img = []

def face_net(image):
    img_cp = cv2.resize(image, (0,0), fx = 0.2, fy = 0.2)
    img_cp = cv2.cvtColor(img_cp, cv2.COLOR_BGR2RGB) 
    a = embedder.extract(img_cp, threshold=0.95) 
    all_faces.append([])
    faces_img.append([])
    faces_img_box.append([])
    for face in a:
      (x, y, w, h) = face['box']
      all_faces[-1].append(face['embedding']) 
      faces_img_box[-1].append((x,y,h,w))
      faces_img[-1].append(img_cp[y:y+h, x:x+w])

video = cv2.VideoCapture(filename)
ret, frame = video.read()
pbar = tqdm(total=int(video.get(cv2.CAP_PROP_FRAME_COUNT)), position=0, leave=True)
while ret:
  face_net(frame)
  pbar.update(1)
  ret, frame = video.read()

video.release()

100%|█████████▉| 5727/5728 [45:24<00:00,  2.76it/s]

In [ ]:
labels = db.labels_
all_faces_together = sum(all_faces, [])
len(all_faces_together)

5529

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.73)
db.fit_predict(all_faces_together)

j=0
labeled_frames = []
labeled_frames_embed = []
for i in range(len(all_faces)):
  labeled_frames.append([])
  labeled_frames_embed.append([])
  for m in range(len(all_faces[i])):
    labeled_frames[-1].append(labels[j])
    labeled_frames_embed[-1].append((labels[j], faces_img_box[i][m]))
    j+=1

In [ ]:
def face_net_out(image):
    global j
    img_cp = cv2.resize(image, (0,0), fx = 0.25, fy = 0.25)
    img_cp = cv2.cvtColor(img_cp, cv2.COLOR_BGR2RGB)
    for i in labeled_frames_embed[j]:
      label, (x,y,w,h) = i
      cv2.rectangle(img_cp, (x,y), (x+w, y+h), (255,128,128))
      cv2.putText(img_cp,str(label), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,(255, 128, 128), 2)
    out.write(img_cp)
    j+=1
      


out = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'H264'), video.get(cv2.CAP_PROP_FPS), size)
video = cv2.VideoCapture(filename)
ret, frame = video.read()
pbar = tqdm(total=int(video.get(cv2.CAP_PROP_FRAME_COUNT)), position=0, leave=True)
j = 0
while ret:
  face_net(frame)
  pbar.update(1)
  ret, frame = video.read()

out.release()
video.release()

## Embedding compare

Here it is pretty much the same as was on the last lab. The only difference is that it uses facenet. 

In [2]:
%%capture
!wget https://i.imgur.com/W7MvTpr.jpg
!wget https://i.imgur.com/pPJV2q2.jpg
!wget https://i.imgur.com/rsB2TrY.jpg
!wget https://i.imgur.com/Ukxvflp.png
!wget https://i.imgur.com/w1B4xMQ.png
!wget https://i.imgur.com/p2z2exf.png
!wget https://i.imgur.com/yl5m8Dr.png
!wget https://i.imgur.com/eyb5tpa.png
!wget https://i.imgur.com/2zeUq5f.png
!wget https://i.imgur.com/60DHx9R.png
!wget https://i.imgur.com/TD59efj.png
!wget https://i.imgur.com/1rOwnWw.png
!wget https://i.imgur.com/7NLxI55.png
!wget https://i.imgur.com/xsHnBmx.png
!wget https://i.imgur.com/Xzh7Ba6.png
!wget https://i.imgur.com/HH9fAQw.png
!wget https://i.imgur.com/zK9RspG.png
!wget https://i.imgur.com/hHa1ZVH.png
!wget https://i.imgur.com/5FUvO2O.png
!wget https://i.imgur.com/BoYkdRE.png
!wget https://i.imgur.com/MIzLtvu.png
!wget https://i.imgur.com/Tb25BFa.png
!wget https://i.imgur.com/Bllyd3g.png
!wget https://i.imgur.com/sipFF3Z.png
!wget https://i.imgur.com/FXVlHJy.png
!wget https://i.imgur.com/NLIG8Fp.png
!wget https://i.imgur.com/6VgwVUH.png
!wget https://i.imgur.com/QOSZy7g.png

In [3]:
import cv2
import matplotlib.pyplot as plt
import face_recognition
import numpy as np
from keras_facenet import FaceNet
from tqdm import tqdm

filename = "YOLO - We Are One ft. Культура Небес (Official Music Video)-gOFt-sz1T2A.webm"

In [4]:
embedder = FaceNet()
def encode_faces(img):
  faces = embedder.extract(img)
  return [embed['embedding'] for embed in faces], [embed['box'] for embed in faces]

faces_paths={}
faces_paths['Zuhair'] = ['W7MvTpr.jpg','60DHx9R.png','1rOwnWw.png','MIzLtvu.png',
                   'Tb25BFa.png','6VgwVUH.png']
faces_paths['Rafik'] = ['pPJV2q2.jpg','HH9fAQw.png']
faces_paths['Andres'] = ['rsB2TrY.jpg','2zeUq5f.png']
faces_paths['Little Boy'] = ['Ukxvflp.png', 'w1B4xMQ.png','NLIG8Fp.png']
faces_paths['Drums Guy'] = ['p2z2exf.png', 'yl5m8Dr.png']
faces_paths['Baye'] = ['w1B4xMQ.png','NLIG8Fp.png']
faces_paths['Little Girl'] = ['TD59efj.png','7NLxI55.png','xsHnBmx.png']
faces_paths['Mother'] = ['Xzh7Ba6.png']
faces_paths['Andrey'] = ['hHa1ZVH.png','5FUvO2O.png','BoYkdRE.png']
faces_paths['Preacher Man'] = ['Bllyd3g.png','sipFF3Z.png']
faces_paths['Arina'] = ['FXVlHJy.png']

                        
face_embeddings = {}
for k,v in faces_paths.items():
  face_embeddings[k]=[]
  for face_p in v:
    face_img = cv2.cvtColor(cv2.imread(face_p), cv2.COLOR_BGR2RGB)
    face_embeddings[k].append(encode_faces(face_img)[0][0])

In [19]:
def get_name(image):
  img_cp = cv2.resize(image, (0,0), fx = size_f, fy = size_f)
  img_cp = cv2.cvtColor(img_cp, cv2.COLOR_BGR2RGB) 
  ret = []
  embeds, locations = encode_faces(img_cp)
  if locations:
    faces_on_image = {}
    for embed, locate in zip(embeds, locations):
      for name, face in face_embeddings.items():
        on_image = face_recognition.compare_faces(face, embed, tolerance = 0.8)
        if True in on_image:
          faces_on_image[name] = locate

    for (x, y, w, h) in locations:
      cv2.rectangle(img_cp, (x, y), (x+w, y+h), (255,128, 128), 4)
    for name,(x, y, w, h) in faces_on_image.items():
      cv2.rectangle(img_cp, (x, y), (x+w, y+h), (255, 128, 128), 4)
      cv2.putText(img_cp,name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,(255, 128, 128), 2)

    ret = faces_on_image.keys()
  out.write(cv2.cvtColor(img_cp, cv2.COLOR_BGR2RGB))
  return ret

video =cv2.VideoCapture(filename)
size_f = 0.3
fps = video.get(cv2.CAP_PROP_FPS)
size = (int(size_f*video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(size_f*video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
pbar = tqdm(total=int(video.get(cv2.CAP_PROP_FRAME_COUNT)), position=0, leave=True)
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, size)
ret, frame = video.read()
second = 0
second_names={}
frame_count = 0
while ret:
  names = get_name(frame)
  ret, frame = video.read()
  if frame_count%fps == 0:
    second+=1
  for n in names:
    second_names.setdefault(n,set()).add(second)
  frame_count+=1
  pbar.update(1)
video.release()
out.release()

100%|█████████▉| 5727/5728 [53:22<00:00,  2.43it/s]

In [20]:
import pandas as pd

names_dict = []
for name in second_names.keys():
  secs = sorted(list(second_names[name]))
  secs_format = []
  begin, end = 0,-1
  for i in range(len(secs)):
    if end==-1:
      begin = secs[i]
      end = secs[i]  
    else:
      if secs[i] - end>1:
        secs_format.append(f"{begin//60}:{begin%60}s-{end//60}:{end%60}s")
        begin = secs[i]
        end = secs[i]
      if secs[i] - end==1:
        end = secs[i]
  names_dict.append({}) 
  names_dict[-1]['name'] = name
  names_dict[-1]['image'] = './'+faces_paths[name][0]
  names_dict[-1]['timestamps']=", ".join(secs_format)



In [23]:
df = pd.DataFrame(names_dict)

In [24]:
from IPython.core.display import display, HTML

def path_to_image_html(path):
  return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

#Images don't show for som
display(HTML(df.to_html(escape=False ,formatters=dict(image=path_to_image_html))))

,name,image,timestamps
0,Andres,,"0:3s-0:4s, 0:22s-0:23s, 0:31s-0:31s, 0:33s-0:35s, 0:48s-0:49s, 1:0s-1:1s, 1:6s-1:7s, 1:10s-1:10s, 1:15s-1:15s, 2:36s-2:36s, 2:38s-2:38s, 3:10s-3:10s"
1,Drums Guy,,"0:17s-0:19s, 0:22s-0:22s, 1:45s-1:45s, 2:5s-2:10s, 2:17s-2:19s, 2:27s-2:27s, 2:32s-2:33s, 2:35s-2:38s, 3:6s-3:7s, 3:9s-3:11s"
2,Little Boy,,"0:22s-0:23s, 0:29s-0:31s, 1:1s-1:2s, 1:37s-1:43s, 1:51s-1:52s, 1:59s-2:2s, 2:4s-2:6s, 2:9s-2:10s, 2:36s-2:36s, 2:40s-2:42s, 2:47s-2:48s, 2:54s-2:55s, 2:57s-2:57s, 3:10s-3:11s, 3:14s-3:16s"
3,Baye,,"0:22s-0:23s, 0:29s-0:31s, 1:1s-1:2s, 1:37s-1:37s, 2:2s-2:2s, 2:4s-2:6s, 2:9s-2:10s, 2:36s-2:36s"
4,Zuhair,,"0:35s-0:38s, 0:49s-0:50s, 1:2s-1:2s, 1:12s-1:17s, 1:27s-1:29s, 1:36s-1:38s, 1:43s-1:45s, 2:1s-2:1s, 2:4s-2:7s, 2:9s-2:10s, 2:37s-2:40s, 2:43s-2:47s, 2:51s-2:51s, 2:54s-2:54s, 2:57s-3:1s, 3:3s-3:4s"
5,Rafik,,"0:35s-0:37s, 0:48s-0:49s, 1:0s-1:5s, 1:11s-1:12s, 1:14s-1:15s, 1:20s-1:24s, 1:26s-1:27s, 1:29s-1:31s, 1:33s-1:34s, 1:37s-1:38s, 1:43s-1:44s, 1:49s-1:49s, 2:3s-2:7s, 2:9s-2:10s, 2:37s-2:39s, 3:21s-3:22s"
6,Little Girl,,"0:43s-0:46s, 0:50s-0:51s"
7,Mother,,
8,Preacher Man,,"2:2s-2:3s, 2:7s-2:12s, 2:14s-2:17s, 2:21s-2:27s, 2:30s-2:31s, 2:33s-2:38s, 3:6s-3:8s, 3:10s-3:11s"
9,Andrey,,"2:6s-2:7s, 2:9s-2:10s, 2:12s-2:14s, 2:18s-2:22s, 2:24s-2:28s"
